In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_curve, roc_auc_score, confusion_matrix

In [3]:
df = pd.read_csv('churn_data.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [5]:
df.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
df = pd.get_dummies(df, columns=['Geography', 'Gender'])
df.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,1,15634602,Hargrave,619,42,2,0.00,1,1,1,101348.88,1,1,0,0,1,0
1,2,15647311,Hill,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1,1,0
2,3,15619304,Onio,502,42,8,159660.80,3,1,0,113931.57,1,1,0,0,1,0
3,4,15701354,Boni,699,39,1,0.00,2,0,0,93826.63,0,1,0,0,1,0
4,5,15737888,Mitchell,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1,1,0


In [7]:
scaler = StandardScaler()
cols = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
scaled = scaler.fit_transform(df[cols])
df[cols] = scaled
df.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
count,10000.00000,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,10000.00000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,-4.824585e-16,2.318146e-16,-1.078249e-16,-6.252776e-17,1.634248e-17,0.70550,0.515100,-2.877698e-17,0.203700,0.501400,0.250900,0.247700,0.454300,0.545700
std,2886.89568,7.193619e+04,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,0.45584,0.499797,1.000050e+00,0.402769,0.500023,0.433553,0.431698,0.497932,0.497932
min,1.00000,1.556570e+07,-3.109504e+00,-1.994969e+00,-1.733315e+00,-1.225848e+00,-9.115835e-01,0.00000,0.000000,-1.740268e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2500.75000,1.562853e+07,-6.883586e-01,-6.600185e-01,-6.959818e-01,-1.225848e+00,-9.115835e-01,0.00000,0.000000,-8.535935e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5000.50000,1.569074e+07,1.522218e-02,-1.832505e-01,-4.425957e-03,3.319639e-01,-9.115835e-01,1.00000,1.000000,1.802807e-03,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
75%,7500.25000,1.575323e+07,6.981094e-01,4.842246e-01,6.871299e-01,8.199205e-01,8.077366e-01,1.00000,1.000000,8.572431e-01,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000
max,10000.00000,1.581569e+07,2.063884e+00,5.061197e+00,1.724464e+00,2.795323e+00,4.246377e+00,1.00000,1.000000,1.737200e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
def get_metrics(y_test, y_score, b=1):
    metrics = {}
    
    precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
    fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)
    ix = np.argmax(fscore)
    
    metrics['threshold'] = thresholds[ix]
    metrics['fscore'] = fscore[ix]
    metrics['precision'] = precision[ix]
    metrics['recall'] = recall[ix]
    metrics['roc_auc'] = roc_auc_score(y_test, y_score)
    return metrics

def run_classifier(clf, X_train, X_test, y_train, y_test):
    clf.fit(X_train, y_train)
    y_score = clf.predict_proba(X_test)[:, 1]
    return y_score, get_metrics(y_test, y_score)

In [9]:
drop_cols = ['Surname', 'RowNumber', 'CustomerId', 'Exited']
X_train, X_test, y_train, y_test = train_test_split(df.drop(drop_cols, axis=1),
                                                    df['Exited'], random_state=0)

In [10]:
clfs = {}
clfs['LogisticRegression'] = LogisticRegression(random_state=42)
clfs['RandomForestClassifier'] = RandomForestClassifier(max_depth=7, random_state=42)
clfs['KNeighborsClassifier'] = KNeighborsClassifier(n_neighbors=10)
clfs['GradientBoostingClassifier'] = GradientBoostingClassifier(n_estimators=300, max_depth=7)

results = []
max_roc_auc = 0
for name, clf in clfs.items():
    proba, metrics = run_classifier(clf, X_train, X_test, y_train, y_test)
    
    d = {'classifier': name}
    d.update(metrics)
    results.append(pd.DataFrame(d, index=[0]))
    
    if metrics['roc_auc'] > max_roc_auc:
        max_roc_auc = metrics['roc_auc']
        best_proba = proba
        best_metrics = metrics
    
pd.concat(results, ignore_index=True).sort_values(by='roc_auc', ascending=False)

,classifier,threshold,fscore,precision,recall,roc_auc
1,RandomForestClassifier,0.308670,0.636719,0.633010,0.640472,0.865019
3,GradientBoostingClassifier,0.482754,0.621622,0.728232,0.542240,0.853293
2,KNeighborsClassifier,0.400000,0.588123,0.573832,0.603143,0.820844
0,LogisticRegression,0.254466,0.505529,0.422721,0.628684,0.772029


In [11]:
cnf_matrix = confusion_matrix(y_test, best_proba > best_metrics['threshold'])
TP = cnf_matrix[1][1]
FP = cnf_matrix[0][1]

# сделать оценку экономической эффективности (1 доллар на привлечение, 2 доллара - с каждого правильно классифицированного удержанного)
2*TP - 1*(TP + FP)

136